In [155]:
import numpy as np
import cv2
import os
import random
import torch
from torch.autograd import Variable
from torchvision import transforms
output_path = "mnist_train_images/"

In [156]:
data = np.genfromtxt("mnist_train.csv", delimiter=',', skip_header=1)
labels = data[:, 0].astype(np.uint8)
values = data[:,1:].astype(np.uint8)
images = np.reshape(values, (-1,28,28))


In [157]:
test_data = np.genfromtxt("mnist_test.csv", delimiter=',', skip_header=1)
test_values = test_data[:,1:].astype(np.uint8)
test_labels = test_data[:,0].astype(np.uint8)
test_images = np.reshape(test_values, (-1,28,28))

In [34]:

# os.makedirs(output_path, exist_ok=True)
# for i in range(len(images)):
#     filename = os.path.join(output_path, f'{i}.jpg')
#     cv2.imwrite(filename, images[i])
# images1 = []
# images_path = [os.path.join(output_path, f) for f in os.listdir(output_path) if f.endswith('.jpg')]
# for i, path in enumerate(images_path):
#     temp = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
#     if i != 60000:
#         images1.append(temp)

In [38]:
# images1 = np.asarray(images1)

In [158]:
learning_rate = 0.01
batch_size = 200
epochs = 10
log_interval = 10

In [159]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [160]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(28*28, 200)
        self.fc2 = nn.Linear(200, 200)
        self.fc3 = nn.Linear(200, 10)
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return F.log_softmax(x)


In [161]:
net = Net()
net

Net(
  (fc1): Linear(in_features=784, out_features=200, bias=True)
  (fc2): Linear(in_features=200, out_features=200, bias=True)
  (fc3): Linear(in_features=200, out_features=10, bias=True)
)

In [162]:
optimizer = optim.SGD(net.parameters(), lr = learning_rate, momentum=0.9)
criterion = nn.NLLLoss()

In [163]:
def rot_mat(deg):
   theta = deg/ 180 * np.pi
   c,s = np.cos(theta),np.sin(theta)
   return np.array([[c,-s],[s,c]])

In [164]:
def rotate(im, label):
    degree = random.randint(1,180)
    if (label == 6 or label == 9) and degree == 90:
        while degree != 90:
            degree = random.randint(1, 180)
    h, w, c = im.shape
    h2, w2 = h // 2, w // 2
    wr2, hr2 = (np.max(np.abs(rot_mat(degree) @ np.array([[-w2, w2, w2], [h2, h2, -h2]])), axis=1)).astype(np.int32)
    wr, hr = wr2 * 2, hr2 * 2
    imr = np.zeros((wr, hr, 3))
    yr, xr = np.indices((hr, wr))
    yr, xr = yr.flatten(), xr.flatten()
    yrc, xrc = yr - hr2, xr - wr2
    xc, yc = (rot_mat(-degree) @ np.row_stack((xrc, yrc))).astype(np.int32)
    x, y = xc + w2, yc + h2
    include = np.logical_and(np.abs(xc) < w2, np.abs(yc) < h2)
    include.mean()
    imr[yr[include], xr[include]] = im[y[include], x[include]]
    return imr

In [165]:
def noise(im):
    noize = np.zeros(im.shape,np.uint8)
    cv2.randn(noize,0,50)
    n_im = cv2.add(im,noize)
    return n_im

In [166]:
def normalize(im):
    imin = float(im.min())
    imax = float(im.max())
    return (im - imin)/(imax - imin)

In [167]:
normalized_images = normalize(images)


In [20]:
t_im= torch.Tensor(np.array(normalized_images))

In [172]:
for epoch in range(epochs):
    for i in range(len(images) // batch_size):
        batch = batch_size * i
        i_batch = images[batch:batch + batch_size]
        l_batch = labels[batch:batch + batch_size]
        for i in range(len(i_batch)):
            i_batch[i] = noise(i_batch[i])
            # There should also be a rotated here, but it needs to be redone
        im_batch = torch.from_numpy(normalize(i_batch)).float()
        lb_batch = torch.from_numpy(l_batch)

        data, target = Variable(im_batch), Variable(lb_batch)
        data = data.view(-1,28*28)
        optimizer.zero_grad()
        net_out = net(data)
        loss = criterion(net_out, target)
        loss.backward()
        optimizer.step()
        if batch % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch * len(data), len(images),
                       100. * batch / len(images), loss.data))

/tmp/ipykernel_30388/3114604308.py:11: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)


Train Epoch: 0 [0/60000 (0%)]	Loss: 0.167322
Train Epoch: 0 [40000/60000 (0%)]	Loss: 0.130186
Train Epoch: 0 [80000/60000 (1%)]	Loss: 0.142238
Train Epoch: 0 [120000/60000 (1%)]	Loss: 0.162844
Train Epoch: 0 [160000/60000 (1%)]	Loss: 0.101778
Train Epoch: 0 [200000/60000 (2%)]	Loss: 0.204272
Train Epoch: 0 [240000/60000 (2%)]	Loss: 0.176244
Train Epoch: 0 [280000/60000 (2%)]	Loss: 0.114023
Train Epoch: 0 [320000/60000 (3%)]	Loss: 0.085855
Train Epoch: 0 [360000/60000 (3%)]	Loss: 0.092458
Train Epoch: 0 [400000/60000 (3%)]	Loss: 0.134204
Train Epoch: 0 [440000/60000 (4%)]	Loss: 0.045024
Train Epoch: 0 [480000/60000 (4%)]	Loss: 0.054769
Train Epoch: 0 [520000/60000 (4%)]	Loss: 0.162603
Train Epoch: 0 [560000/60000 (5%)]	Loss: 0.098112
Train Epoch: 0 [600000/60000 (5%)]	Loss: 0.081157
Train Epoch: 0 [640000/60000 (5%)]	Loss: 0.076347
Train Epoch: 0 [680000/60000 (6%)]	Loss: 0.068599
Train Epoch: 0 [720000/60000 (6%)]	Loss: 0.106974
Train Epoch: 0 [760000/60000 (6%)]	Loss: 0.062381
Train E

In [173]:
test_loss = 0
correct = 0

In [174]:
norm_test_values = normalize(test_values)


In [175]:
for i in range(len(norm_test_values)//batch_size):
    batch = batch_size * i
    im_batch = torch.from_numpy(norm_test_values[batch:batch + batch_size]).float()
    lb_batch = torch.from_numpy(test_labels[batch:batch + batch_size])
    data, target = Variable(im_batch, volatile=True), Variable(lb_batch)
    data = data.view(-1,28*28)
    net_out = net(data)
    test_loss += criterion(net_out, target).data
    pred = net_out.data.max(1)[1]
    correct += pred.eq(target.data).sum()
test_loss /= len(test_images)
print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
    test_loss, correct, len(test_images),
    100. * correct / len(test_images)))


Test set: Average loss: 0.0019, Accuracy: 9391/10000 (94%)



/tmp/ipykernel_30388/2908435613.py:5: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  data, target = Variable(im_batch, volatile=True), Variable(lb_batch)
/tmp/ipykernel_30388/3114604308.py:11: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)


In [125]:
# i_batch = 0
# for i in range(len(values) // batch_size):
#     batch = batch_size * i
#     i_batch = images[batch:batch + batch_size]
#     l_batch = labels[batch:batch + batch_size]
#     for i in range(len(i_batch)):
#         i_batch[i] = noise(i_batch[i])
